In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M1','M5','M15','H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc

# Stats

In [54]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)

Hilo                                     
    count mean  std  min  25%  50%   75%   max
TF                                            
M5    905  495  218  122  350  450   585  2267
M15  1246  656  295  191  466  590   771  3209
H1   1246  944  438  241  630  858  1148  3852

In [55]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                     
     count mean  std  min  25%  50%   75%   max
TF                                             
M5     449  495  203  122  354  458   595  1512
M15    609  661  299  191  471  596   762  2812
H1     639  937  422  241  626  861  1144  3236

In [56]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                     
     count mean  std   min   25%  50%  75%  max
TF                                             
M5     456 -495  232 -2267  -577 -438 -350 -163
M15    637 -651  292 -3209  -776 -581 -465 -225
H1     607 -952  453 -3852 -1148 -851 -636 -309

In [57]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                    
     count mean  std   min  25% 50%  75%   max
TF                                            
M5     905   -7  320 -2131 -216  -7  205  1050
M15   1246   -8  420 -1884 -274 -12  232  1647
H1    1246   -7  609 -3113 -378  15  350  2811

In [58]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5      905  250  200   0   97  212  351  2131
M15    1246  320  272   0  114  250  443  1884
H1     1246  461  398   0  164  361  650  3113

In [59]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5      449  245  190   0  100  207  341  1050
M15     609  319  277   0  117  241  448  1647
H1      639  443  387   0  138  344  652  2811

In [60]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
      count mean  std   min  25%  50%  75% max
TF                                            
M5      456 -255  210 -2131 -360 -215  -96  -5
M15     637 -321  266 -1884 -437 -266 -113  -5
H1      607 -480  410 -3113 -646 -383 -187  -5

# Snippets

In [61]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

In [ ]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)